<a href="https://colab.research.google.com/github/imusicmash/stanford_llm_python/blob/main/react_agent_with_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/agent/react_agent_with_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReAct Agent with Query Engine (RAG) Tools

In this section, we show how to setup an agent powered by the ReAct loop for financial analysis.

The agent has access to two "tools": one to query the 2021 Lyft 10-K and the other to query the 2021 Uber 10-K.

We try two different LLMs:

- gpt-3.5-turbo
- gpt-3.5-turbo-instruct

Note that you can plug in any LLM that exposes a text completion endpoint.

## Build Query Engine Tools

In [ ]:
# I built this out after class, as i discovered it here
# from https://docs.llamaindex.ai/en/stable/examples/agent/react_agent_with_query_engine.html

In [2]:
%pip install llama-index-llms-openai

In [14]:
!pip install llama-index --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 111.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from openai import OpenAI
from google.colab import userdata

open_ai_key = userdata.get('openai')
# client = OpenAI(api_key=open_ai_key)

In [5]:
import os
os.environ["OPENAI_API_KEY"] = open_ai_key

In [11]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata


In [12]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

Download Data

In [7]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-03-05 20:11:31--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.06s   

2024-03-05 20:11:32 (28.8 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-03-05 20:11:32--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP 

In [15]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [16]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

In [17]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Setup ReAct Agent

Here we setup two ReAct agents: one powered by standard gpt-3.5-turbo, and the other powered by gpt-3.5-turbo-instruct.

You can **optionally** specify context which will be added to the core ReAct system prompt.

In [18]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [20]:
# [Optional] Add Context
# context = """\
# You are a stock market sorcerer who is an expert on the companies Lyft and Uber.\
#     You will answer questions about Uber and Lyft as in the persona of a sorcerer \
#     and veteran stock market investor.
# """
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [22]:
response = agent.chat("What was Uber's revenue growth in 2021?")
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth in 2021?"}
Observation: Uber's revenue grew by 57% in 2021.
Thought: I can answer without using any more tools.
Answer: Uber's revenue growth in 2021 was 57%.
Uber's revenue growth in 2021 was 57%.


## Run Some Example Queries

We run some example queries using the agent, showcasing some of the agent's abilities to do chain-of-thought-reasoning and tool use to synthesize the right answer.

We also show queries.

In [23]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
print(str(response))

Thought: I need to compare the revenue growth of Uber and Lyft in 2021 to provide an analysis.

Action: lyft_10k
Action Input: {"input": "What was Lyft's revenue growth in 2021?"}

Observation: Lyft's revenue growth in 2021 was 36%.

Action: uber_10k
Action Input: {"input": "What was Uber's revenue growth in 2021?"}

Observation: Uber's revenue growth in 2021 was 57%.

Thought: I have the necessary information to compare and contrast the revenue growth of Uber and Lyft in 2021.
Answer: In 2021, Lyft's revenue growth was 36% while Uber's revenue growth was 57%. This indicates that Uber experienced a higher revenue growth rate compared to Lyft. The higher growth rate for Uber could be attributed to various factors such as a larger market share, expansion into new markets, or successful strategic initiatives. However, without further information, it is difficult to determine the exact reasons for the difference in revenue growth between the two companies.
In 2021, Lyft's revenue growth wa

**Async execution**: Here we try another query with async execution

In [24]:
# Try another query with async execution

import nest_asyncio

nest_asyncio.apply()

response = await agent.achat(
    "Compare and contrast the risks of Uber and Lyft in 2021, then give an"
    " analysis"
)
print(str(response))

Thought: I need to use a tool to help me compare and contrast the risks of Uber and Lyft in 2021.
Action: uber_10k
Action Input: {'input': 'Please provide the risks of Uber in 2021.'}
Observation: The risks faced by Uber in 2021 include challenges related to the impact of the COVID-19 pandemic on their financial results, workforce reductions, changes in pricing models, uncertainty around the pandemic's ultimate impact on business operations, liquidity, and financial condition, as well as the potential for sustained weak demand for their Mobility offering. Additionally, risks include regulatory challenges regarding the classification of Drivers, potential disruptions in operations due to airport regulations and bans on ridesharing services, and the need to successfully offer autonomous vehicle technologies on their platform to remain competitive.
Thought: I need to use a tool to help me compare and contrast the risks of Lyft in 2021.
Action: lyft_10k
Action Input: {'input': 'Please prov

### Compare gpt-3.5-turbo vs. gpt-3.5-turbo-instruct

We compare the performance of the two agents in being able to answer some complex queries.

#### Taking a look at a turbo-instruct agent

In [25]:
llm_instruct = OpenAI(model="gpt-3.5-turbo-instruct")
agent_instruct = ReActAgent.from_tools(
    query_engine_tools, llm=llm_instruct, verbose=True
)

In [26]:
response = agent_instruct.chat("What was Lyft's revenue growth in 2021?")
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue increased by 36% in 2021 compared to the prior year.
Thought: I can answer without using any more tools.
Answer: Lyft's revenue growth in 2021 was 36%.
Lyft's revenue growth in 2021 was 36%.


#### Try more complex queries

We compare gpt-3.5-turbo with gpt-3.5-turbo-instruct agents on more complex queries.

In [ ]:
response = agent.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
print(str(response))

Thought: I need to use a tool to help me compare the revenue growth of Uber and Lyft in 2021.
Action: uber_10k
Action Input: {'input': "Please provide information about Uber's revenue growth in 2021."}
Observation: Uber's revenue grew by 57% in 2021 compared to the previous year. This growth was primarily driven by an increase in Gross Bookings, with Delivery Gross Bookings increasing by 71% and Mobility Gross Bookings growing by 38%. The increase in Delivery Gross Bookings was due to higher demand for food delivery orders and expansion across U.S. and international markets. The growth in Mobility Gross Bookings was a result of increased Trip volumes as the business recovered from the impacts of COVID-19.
Thought: I have information about Uber's revenue growth in 2021. Now I need to use a tool to get information about Lyft's revenue growth in 2021.
Action: lyft_10k
Action Input: {'input': "Please provide information about Lyft's revenue growth in 2021."}
Observation: Lyft's revenue inc

In [ ]:
response = agent_instruct.chat(
    "Compare and contrast the revenue growth of Uber and Lyft in 2021, then"
    " give an analysis"
)
print(str(response))

Response: The revenue growth of Uber was higher than Lyft in 2021, with Uber experiencing a 74% growth compared to Lyft's 48%. This indicates that Uber may have had a stronger financial performance in 2021. However, further analysis is needed to fully understand the factors contributing to this difference.
The revenue growth of Uber was higher than Lyft in 2021, with Uber experiencing a 74% growth compared to Lyft's 48%. This indicates that Uber may have had a stronger financial performance in 2021. However, further analysis is needed to fully understand the factors contributing to this difference.


In [ ]:
response = agent.chat(
    "Can you tell me about the risk factors of the company with the higher"
    " revenue?"
)
print(str(response))

Thought: I need to find out which company has higher revenue before I can provide information about its risk factors.
Action: lyft_10k
Action Input: {'input': 'What is the revenue of Lyft in 2021?'}
Observation: The revenue of Lyft in 2021 is $3,208,323,000.
Thought: Now that I know Lyft has higher revenue, I can find information about its risk factors.
Action: lyft_10k
Action Input: {'input': 'What are the risk factors of Lyft?'}
Observation: Lyft faces numerous risk factors that could potentially harm its business, financial condition, and results of operations. These risk factors include general economic factors such as the impact of the COVID-19 pandemic, natural disasters, economic downturns, and political crises. Operational factors such as limited operating history, financial performance, competition, unpredictability of results, uncertainty regarding market growth, ability to attract and retain drivers and riders, insurance coverage, autonomous vehicle technology, reputation an

In [ ]:
response = agent_instruct.query(
    "Can you tell me about the risk factors of the company with the higher"
    " revenue?"
)
print(str(response))

Response: The risk factors for the company with the higher revenue include competition, regulatory changes, and dependence on drivers.
The risk factors for the company with the higher revenue include competition, regulatory changes, and dependence on drivers.


**Observation**: The turbo-instruct agent seems to do worse on agent reasoning compared to the regular turbo model. Of course, this is subject to further observation!